In [1]:
import pandas as pd
import glob
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))
from config import *

In [2]:
#path to txt files
file_path_cases = output_path+'/cases/*multianno.txt'
#list all txt files
cases = glob.glob(file_path_cases)

file_path_controls = output_path+'/control/*multianno.txt'
control = glob.glob(file_path_controls)

In [4]:
categories = ['exonic', 'UTR3', 'UTR5', 'ncRNA_exonic', 'ncRNA_intronic', 'upstream', 'downstream', 'intronic', 'splicing']
final_results = pd.DataFrame(columns=['patient_id'] + categories + ['label'])

In [ ]:
target_gene = 'NOD2'
for file in cases + control:
    df = pd.read_csv(file, sep="\t", usecols=[0, 1, 2, 3, 4, 5, 6, 8], low_memory=False)
    patient_id = file.split("/")[-1].replace(".hg19_multianno.txt", "")
    function_col = 'Func.refGene'
    genes_col = 'Gene.refGene'
    # Filter for the target gene and non-synonymous variants
    df_filtered = df[df[genes_col].str.contains(r'\b' + target_gene + r'\b', case=False)]
    #df_filtered = df_filtered[df_filtered['ExonicFunc.refGene'] != 'synonymous SNV']
    # Count variants by category
    variant_counts = df_filtered[function_col].value_counts().reindex(categories, fill_value=0)
    # Determine the label (1 for cases, 0 for controls)
    label = 1 if "cases" in file else 0
    # Create a result row
    result_row = pd.DataFrame([[patient_id] + list(variant_counts) + [label]], columns=['patient_id'] + categories + ['label'])
    # Append the result row to the final results
    final_results = pd.concat([final_results, result_row], ignore_index=True)

final_results.to_csv(vcf_path + "/gene_variant_counts.csv", index=False)
print(final_results)

    patient_id exonic UTR3 UTR5 ncRNA_exonic ncRNA_intronic upstream  \
0          CD1      0    0    0            0              0        0   
1         CD10      0    0    0            0              0        0   
2         CD11      0    0    0            0              0        0   
3         CD12      0    0    0            0              0        0   
4         CD13      0    0    0            0              0        0   
..         ...    ...  ...  ...          ...            ...      ...   
106        HC5      0    0    0            0              0        0   
107        HC6      0    0    0            0              0        0   
108        HC7      0    0    0            0              0        0   
109        HC8      0    0    0            0              0        0   
110        HC9      0    0    0            0              0        0   

    downstream intronic splicing label  
0            0        0        0     1  
1            0        0        0     1  
2           